# Module 4: SQLAlchemy ORM Fundamentals

## Learning Objectives
- Master SQLAlchemy ORM concepts and setup
- Learn model definition and relationships
- Practice basic CRUD operations
- Understand database migrations

## Topics Covered
- SQLAlchemy setup and configuration
- Model definition and relationships
- Basic CRUD operations
- Database migrations with Alembic
- Query building and filtering

## Time Estimate: 2-3 weeks
## Success Criteria: Build a complete CRUD application with SQLAlchemy

---

## Section 1: SQLAlchemy Setup and Configuration

SQLAlchemy is a powerful Object-Relational Mapping (ORM) library for Python. It provides a high-level interface for working with databases using Python objects instead of raw SQL.


In [ ]:
# SQLAlchemy Setup and Basic Configuration

from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime
import os

# Database configuration
DATABASE_URL = "sqlite:///sqlalchemy_demo.db"

# Create engine
engine = create_engine(DATABASE_URL, echo=True)  # echo=True shows SQL queries

# Create base class for models
Base = declarative_base()

# Create session factory
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Define models
class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String(50), unique=True, index=True, nullable=False)
    email = Column(String(100), unique=True, index=True, nullable=False)
    first_name = Column(String(50), nullable=False)
    last_name = Column(String(50), nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationship to posts
    posts = relationship("Post", back_populates="author")

class Category(Base):
    __tablename__ = "categories"
    
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String(100), nullable=False)
    description = Column(Text)
    
    # Relationship to posts
    posts = relationship("Post", back_populates="category")

class Post(Base):
    __tablename__ = "posts"
    
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(200), nullable=False)
    content = Column(Text, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # Foreign keys
    author_id = Column(Integer, ForeignKey("users.id"), nullable=False)
    category_id = Column(Integer, ForeignKey("categories.id"), nullable=False)
    
    # Relationships
    author = relationship("User", back_populates="posts")
    category = relationship("Category", back_populates="posts")

# Create all tables
Base.metadata.create_all(bind=engine)

print("✅ SQLAlchemy setup completed!")
print("✅ Database tables created!")
print("\nModels created:")
print("- User (with posts relationship)")
print("- Category (with posts relationship)")
print("- Post (with author and category relationships)")

# Create a session
session = SessionLocal()

# Insert sample data
try:
    # Create users
    user1 = User(
        username="alice_dev",
        email="alice@example.com",
        first_name="Alice",
        last_name="Developer"
    )
    
    user2 = User(
        username="bob_coder",
        email="bob@example.com",
        first_name="Bob",
        last_name="Coder"
    )
    
    session.add_all([user1, user2])
    session.commit()
    
    # Create categories
    category1 = Category(
        name="Python",
        description="Python programming tutorials and tips"
    )
    
    category2 = Category(
        name="Database",
        description="Database design and SQL tutorials"
    )
    
    session.add_all([category1, category2])
    session.commit()
    
    # Create posts
    post1 = Post(
        title="Getting Started with Python",
        content="Python is a powerful and versatile programming language...",
        author_id=user1.id,
        category_id=category1.id
    )
    
    post2 = Post(
        title="SQLAlchemy ORM Basics",
        content="SQLAlchemy is a powerful ORM for Python...",
        author_id=user2.id,
        category_id=category2.id
    )
    
    session.add_all([post1, post2])
    session.commit()
    
    print("\n✅ Sample data inserted successfully!")
    
except Exception as e:
    session.rollback()
    print(f"Error inserting data: {e}")
finally:
    session.close()
